# Diferenciální rovnice 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp

## Lov v logistické rovnici




### Konstantní užitek

V logistické rovnici nakreslíme pro tři různé intenzity lovu průběh řešení. Prorovnejte následující kód s kódem pro kreslení řešení jedné sady pro jednu intenzitu lovu.

Zpravidla netriviální kód nenapíšeme napoprvé, ale musíme příkazy ladit. V následujícím jsou rozděleny fáze řešení rovnice a vykreslení řešení. Pro potřeby spouštění je po odladění vhodné buňky sloučit. 

* Zkuste si v následující buňce rozdělit kód do dvou různých buněk. Tedy přepnete se do editace, najdete vhodný řádek a v menu vyberete "Edit" a "Split Cell". 
* Poté zkuste buňky co se mají spouštět společně spojit. V příkazovém modu buňky označte (například shift + šipka nahoru nebo dolů) a stisknout velké M, tj. Shift + M. Pozor, pokud byste stiskli malé "m", buňka by se změnila na Markdown buňku s textem. Zpět na buňku s kódem je klávesa "y".

In [ ]:
pocatecni_podminka = np.linspace(0.1,1.2,50)
meze = [0,10]
t = np.linspace(*meze,100)

def destrukce_populace(t,x):  # Pokud x klesne na nulu, zastavíme výpočet
    return x
destrukce_populace.terminal = True

def rovnice(t, x, r=1, K=1, h=0.15):
    return r*x*(1-x/K)-h

lovy = [0.1,0.2,0.3]
# Pro různé počáteční podmínky se bude lišit interval, 
# na kterém algoritmus najde řešení. Proto nemůžeme data
# shrnout do jedné tabulky. Alternativou je tabulka s 
# nedefinovanými hodnotami, viz
# https://robert-marik.github.io/dmp/snippety/tabulky_none.html
# a https://robert-marik.github.io/dmp/snippety/multiindex.html
reseni = [
        [ solve_ivp(
                   lambda t,x:rovnice(t,x,h=h),
                   meze,
                   [pp],
                   t_eval=t,
                   events=destrukce_populace,  # zastavení výpočtu při poklesu populace na nulu
                   )
          for pp in pocatecni_podminka]
        for h in lovy]

# GRAFICKA PREZENTACE VYSLEDKU
fig,ax = plt.subplots()
for i,r in enumerate(reseni):
    for res in r:
        ax.plot(res.t,res.y[0], color=f"C{i}", alpha=0.5, label=f"lov {lovy[i]}", lw=0.5)
ax.set(
  title="Logistická rovnice s konstantním lovem",
  xlabel="bezrozměrný čas",
  ylabel="bezrozměrná velikost populace"
  );

# Návod jak seskupit položky legendy je na https://stackoverflow.com/questions/26337493/pyplot-combine-multiple-line-labels-in-legend
handles, labels = ax.get_legend_handles_labels()
labels, ids = np.unique(labels, return_index=True)
handles = [handles[i] for i in ids]
plt.legend(handles, labels);

In [ ]:
fig,ax = plt.subplots()
x = np.linspace(0,1)
plt.plot(x,rovnice(0, x,h=0),label="růst bez lovu",color='r')
for h in lovy: 
    plt.plot(x,h+0*x,label=f"lov {h}")
ax.set(
    xlabel="velikost populace",
    ylabel="rychlost růstu a lovu",
    title="Srovnání dynamiky růstu a lovu"
)    
plt.legend()    

### Konstantní úsilí

Modifikujte předchozí kód s vykreslením časového vývoje populace vystavené lovu.
Lov konstantní intenzitou nahraďte lovem s konstantním úsilím. Zkuste nejprve minimální úprava kódu.
Bez ohledu na efektivitu, vyjděte z předchozího a snažte se co nejméně
modifikovat výchozí kód. Poté si prostudujte elegantnější přístup využívající
toho, že žádné řešení neskončí v konečném čase.

In [ ]:
# sem napiste reseni

U konstantního úsilí není problém s tím, že by některá řešení končila dříve. Proto může být programový kód kratší a čistý. Například nemusíme pracovat s vnořenými cykly a můžeme příkazu `solve_ivp` poslat současně všechny počáteční podmínky. Pokusme se o to.

In [ ]:
pocatecni_podminka = np.linspace(0.1,1.2,51).round(3)
meze = [0,10]
t = np.linspace(*meze,100)

def rovnice(t, x, r=1, K=1, h=0.15):
    return r*x*(1-x/K)-h*x

lovy = [0.1,0.3,0.6]

### Definice tabulky s víceúrovňovými nadpisy sloupců, MultiIndex
### https://pandas.pydata.org/docs/user_guide/advanced.html
iterables = [lovy,pocatecni_podminka]
my_index = pd.MultiIndex.from_product(iterables, names=['lov', 'poč.podm.'])
df = pd.DataFrame(columns=my_index)
df["čas"] = t
for h in lovy:
    r = solve_ivp(
              lambda t,x:rovnice(t,x,h=h),
              meze,
              pocatecni_podminka,
              t_eval=t,
              ).y.T
    df[[(h,i) for i in pocatecni_podminka]]=pd.DataFrame(r)

df.set_index("čas", inplace=True)
df.T

In [ ]:
fig,ax = plt.subplots()
for i in range(len(lovy)): # tři čáry mimo obrázek kvůli legendě
    ax.plot([0,1],[-1,-1],label=f"none")
for i,h in enumerate(lovy):
    ax.plot(df.index,df[h], color=f"C{i}", alpha=0.5)
ax.set(
  title="Logistická rovnice s lovem s konstantním úsilím",
  xlabel="bezrozměrný čas",
  ylabel="bezrozměrná velikost populace",
  ylim=[0,None]
  )

plt.legend([f"lov {lov}" for lov in lovy]);

In [ ]:
fig,ax = plt.subplots()
x = np.linspace(0,1)
plt.plot(x,rovnice(0, x,h=0),label="růst bez lovu")
for a in lovy: 
    plt.plot(x,a*x,label=f"lov {a}")
ax.set(
    xlabel="velikost populace",
    ylabel="rychlost růstu a lovu",
    title="Srovnání dynamiky růstu a lovu",
    ylim=(0,0.3)
)    
plt.legend() 

## Alleeho efekt

Nakreslete model řešení rovnice modelující lov konstantním úsilím v populaci s
Alleeho efektem. Můžete uvažovat slabý Aleeho efekt (pro malé velikosti
populace se dynamika růstu výrazně zpomalí) nebo silný Aleeho efekt  (pro malé
velikosti populace vymírá). Můžete použít například rovnici
$$\frac{\mathrm dx}{\mathrm dt} = rx ^k\left(1-\frac xK\right) -ax, \quad k>1$$ 
pro slabý nebo 
$$\frac{\mathrm dx}{\mathrm dt} = rx \left(1-\frac xK\right)\left(\frac
xA-1\right) -ax, \quad 0<A<K$$ 
pro silný Alleeho efekt. Nejprve si nakreslete křivky růstu a lovu, pro
vytipování vhodných numerických hodnot pro parametry a poté nakreslete řešení
diferenciální rovnice pro různé intenzity lovu a různé počáteční podmínky. 

In [ ]:
# sem napiste kod pro vykreslení krivek rustu a lovu

In [ ]:
# sem napiste kod pro vykreslení reseni diferenciální rovnice

## Populace pod predačním tlakem

Vykreslete model pro populaci pro predačním tlakem <https://robert-marik.github.io/dmp/prednaska/05.html#populace-pod-predacnim-tlakem>. Použijte bezrozměnrou formulaci, tj. rovnici  $$\frac{ \mathrm dx}{ \mathrm d\tau}=\alpha\left(1-\frac {x}{\beta}\right)x-\frac
   {x^2}{x^2+1}.$$
Tři dvojice hodnot pro $\alpha$ a $\beta$ můžete použít z <https://robert-marik.github.io/dmp/prednaska/05.html#rustove-krivky>.

In [ ]:
# sem napiste reseni